In [34]:
import pandas as pd
import os

# Load CSV
df = pd.read_csv('../data/sub_reg_nt_csvs_consolidados/df_consolidado_51.csv')
df


,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt
0,2000,1,105,0,0,0,0,0,0,51
1,2000,2,58,0,0,0,0,0,0,51
2,2000,3,54,0,0,0,0,0,0,51
3,2000,4,44,0,0,0,0,0,0,51
4,2000,6,65,1,0,0,0,0,0,51
...,...,...,...,...,...,...,...,...,...,...
1231,2023,48,1265,12,20,1,5,0,0,51
1232,2023,49,1026,16,20,10,6,0,1,51
1233,2023,50,1262,19,20,8,6,0,2,51
1234,2023,51,1193,13,15,3,3,0,1,51


In [35]:
# Crear semana absoluta
df['semana_abs'] = df['ano'] * 52 + df['semana']

# Diferencia entre semanas consecutivas
df['diff'] = df['semana_abs'].diff()

# Identificar grupos de continuidad
df['grupo'] = (df['diff'] != 1).cumsum()

# Contar tamaño de cada grupo
secuencias = df.groupby('grupo').size()

# Encontrar el grupo con la secuencia más larga
grupo_max = secuencias.idxmax()
max_secuencia = secuencias.max()

# Obtener los valores iniciales y finales de esa secuencia
sub_df = df[df['grupo'] == grupo_max]
inicio_ano, inicio_semana = sub_df.iloc[0][['ano', 'semana']]
fin_ano, fin_semana = sub_df.iloc[-1][['ano', 'semana']]

# Mostrar resultados
print(f"La secuencia más larga de semanas consecutivas (con años consecutivos incluidos) es de {max_secuencia} semanas.")
print(f"Empieza en el año {inicio_ano}, semana {inicio_semana}, y termina en el año {fin_ano}, semana {fin_semana}.")

# Eliminar columnas auxiliares
df.drop(columns=['semana_abs', 'diff', 'grupo'], inplace=True)

La secuencia más larga de semanas consecutivas (con años consecutivos incluidos) es de 918 semanas.
Empieza en el año 2003.0, semana 19.0, y termina en el año 2020.0, semana 52.0.


Creación de lags

In [36]:
df.size

12360

In [37]:
df.tail(10)

,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt
1226,2023,43,1293,19,20,7,4,0,0,51
1227,2023,44,1117,12,11,5,4,0,0,51
1228,2023,45,1335,9,18,2,3,0,0,51
1229,2023,46,1426,20,17,8,2,0,0,51
1230,2023,47,1284,10,15,2,2,0,1,51
1231,2023,48,1265,12,20,1,5,0,0,51
1232,2023,49,1026,16,20,10,6,0,1,51
1233,2023,50,1262,19,20,8,6,0,2,51
1234,2023,51,1193,13,15,3,3,0,1,51
1235,2023,52,1143,12,14,5,2,0,1,51


In [38]:
print(f"\nValores nulos por columna (primeras columnas con nulos):")
print(df.isnull().sum()[df.isnull().sum() > 0])


Valores nulos por columna (primeras columnas con nulos):
Series([], dtype: int64)


In [39]:
# Columnas a verificar
cols_ffill = [
    'neumonias_men5',
    'neumonias_60mas',
    'hospitalizados_men5',
    'hospitalizados_60mas',
    'defunciones_men5',
    'defunciones_60mas'
]

# Verificar cuáles de esas columnas existen en el DataFrame
cols_existentes = [col for col in cols_ffill if col in df.columns]

# Aplicar fill forward solo si hay nulos
if df[cols_existentes].isnull().any().any():
    df[cols_existentes] = df[cols_existentes].ffill()
    print("Se aplicó forward fill a las columnas con valores nulos.")
else:
    print("No se encontraron valores nulos en las columnas especificadas.")


No se encontraron valores nulos en las columnas especificadas.


In [40]:
import pandas as pd
import numpy as np

# Crear el dataframe con los datos proporcionados


# Ordenar por año y semana
df = df.sort_values(['ano', 'semana']).reset_index(drop=True)

# Columnas para crear lags
columnas_lag = ['ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas', 
                'hospitalizados_men5', 'hospitalizados_60mas', 
                'defunciones_men5', 'defunciones_60mas']

# Crear lags de 1 a 6 semanas y rellenar con 0 los valores nulos
for col in columnas_lag:
    for lag in range(1, 7):
        df[f'{col}_lag_{lag}_semana'] = df[col].shift(lag).fillna(0)

# Función para obtener el mes basado en la semana del año
def semana_a_mes(semana):
    """Convierte semana del año a mes (aproximado)"""
    if semana <= 4:
        return 1
    elif semana <= 8:
        return 2
    elif semana <= 13:
        return 3
    elif semana <= 17:
        return 4
    elif semana <= 21:
        return 5
    elif semana <= 26:
        return 6
    elif semana <= 30:
        return 7
    elif semana <= 35:
        return 8
    elif semana <= 39:
        return 9
    elif semana <= 43:
        return 10
    elif semana <= 48:
        return 11
    else:
        return 12

# Crear características temporales
df['mes'] = df['semana'].apply(semana_a_mes)

# Bimestre (cada 2 meses)
df['bimestre'] = ((df['mes'] - 1) // 2) + 1

# Trimestre (cada 3 meses)
df['trimestre'] = ((df['mes'] - 1) // 3) + 1

# Cuatrimestre (cada 4 meses)
df['cuatrimestre'] = ((df['mes'] - 1) // 4) + 1

# Semestre (cada 6 meses)
df['semestre'] = ((df['mes'] - 1) // 6) + 1

# Pandemia COVID (años 2019-2021)
df['pandemia_covid'] = df['ano'].apply(lambda x: 1 if 2019 <= x <= 2021 else 0)

# Mostrar el resultado
print("DataFrame con lags y características temporales:")
df.head(10)
print("\nColumnas creadas:")
print(df.columns.tolist())
print(f"\nTotal de columnas: {len(df.columns)}")


DataFrame con lags y características temporales:

Columnas creadas:
['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas', 'hospitalizados_men5', 'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas', 'sub_reg_nt', 'ira_no_neumonia_lag_1_semana', 'ira_no_neumonia_lag_2_semana', 'ira_no_neumonia_lag_3_semana', 'ira_no_neumonia_lag_4_semana', 'ira_no_neumonia_lag_5_semana', 'ira_no_neumonia_lag_6_semana', 'neumonias_men5_lag_1_semana', 'neumonias_men5_lag_2_semana', 'neumonias_men5_lag_3_semana', 'neumonias_men5_lag_4_semana', 'neumonias_men5_lag_5_semana', 'neumonias_men5_lag_6_semana', 'neumonias_60mas_lag_1_semana', 'neumonias_60mas_lag_2_semana', 'neumonias_60mas_lag_3_semana', 'neumonias_60mas_lag_4_semana', 'neumonias_60mas_lag_5_semana', 'neumonias_60mas_lag_6_semana', 'hospitalizados_men5_lag_1_semana', 'hospitalizados_men5_lag_2_semana', 'hospitalizados_men5_lag_3_semana', 'hospitalizados_men5_lag_4_semana', 'hospitalizados_men5_lag_5_semana', 'hospi

In [41]:
print(f"\nValores nulos por columna (primeras columnas con nulos):")
print(df.isnull().sum()[df.isnull().sum() > 0])


Valores nulos por columna (primeras columnas con nulos):
Series([], dtype: int64)


In [42]:
df.tail(20)

,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt,...,defunciones_60mas_lag_3_semana,defunciones_60mas_lag_4_semana,defunciones_60mas_lag_5_semana,defunciones_60mas_lag_6_semana,mes,bimestre,trimestre,cuatrimestre,semestre,pandemia_covid
1216,2023,33,1258,18,22,5,5,0,2,51,...,0.0,1.0,1.0,2.0,8,4,3,2,2,0
1217,2023,34,1267,18,16,6,1,0,0,51,...,2.0,0.0,1.0,1.0,8,4,3,2,2,0
1218,2023,35,1477,16,17,4,7,0,2,51,...,2.0,2.0,0.0,1.0,8,4,3,2,2,0
1219,2023,36,1558,15,18,5,1,0,0,51,...,2.0,2.0,2.0,0.0,9,5,3,3,2,0
1220,2023,37,1356,20,29,6,3,0,1,51,...,0.0,2.0,2.0,2.0,9,5,3,3,2,0
1221,2023,38,1375,20,21,2,4,0,1,51,...,2.0,0.0,2.0,2.0,9,5,3,3,2,0
1222,2023,39,1475,20,24,8,6,0,1,51,...,0.0,2.0,0.0,2.0,9,5,3,3,2,0
1223,2023,40,1558,13,27,3,5,0,2,51,...,1.0,0.0,2.0,0.0,10,5,4,3,2,0
1224,2023,41,1557,15,25,5,3,0,2,51,...,1.0,1.0,0.0,2.0,10,5,4,3,2,0
1225,2023,42,1452,15,21,2,0,0,0,51,...,1.0,1.0,1.0,0.0,10,5,4,3,2,0


In [43]:
df.size

71688

In [44]:
# Guardar a CSV en la ruta especificada
sub_reg_value = df['sub_reg_nt'].iloc[0]  # Obtener el valor de sub_reg_nt
output_path = f'../data/03_casos/df_{sub_reg_value}_cleaned.csv'
df.to_csv(output_path, index=False)
print(f"\nArchivo guardado en: {output_path}")


Archivo guardado en: ../data/03_casos/df_51_cleaned.csv
